# Assignment 1: Camera Geometric Calibration
### Developed by: Ana San Roman Gaitero and Jose Luis Cano-Manuel Claver
### 19-02-2023

In [ ]:
import numpy as np
import cv2 as cv
import glob2 as glob
import functions

In [ ]:
# function needs to be on the main script due to the setMouseCallback function
def click_corner(click, x1, y1, flags, params):
    """
    This function displays the coordinates of the points clicked
    on the image, and saves them on a list to later use.
    :param click: click mouse event on the screen
    :param x1: x coordinate
    :param y1: y coordinate
    :param params: for any additional variables
    """
    # checking for left mouse clicks
    if click == cv.EVENT_LBUTTONDOWN:
        # save on list x,y coordinates for the later interpolation
        corner_clicks.append([x1,y1])

        # displaying the coordinates on the image window
        font = cv.FONT_HERSHEY_SIMPLEX
        cv.putText(img, str(x1) + ',' +
                    str(y1), (x1,y1), font,
                    1, (255, 0, 0), 2)
        cv.imshow('img', img)

    # checking for right mouse clicks
    if click==cv.EVENT_RBUTTONDOWN:
        # save on list x,y coordinates for the later interpolation
        corner_clicks.append([x1,y1])
        # displaying the coordinates on the image window
        font = cv.FONT_HERSHEY_SIMPLEX
        b = img[y1, x1, 0]
        g = img[y1, x1, 1]
        r = img[y1, x1, 2]
        cv.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x1,y1), font, 1,
                    (255, 255, 0), 2)
        cv.imshow('img', img)

# Offline

In [ ]:
# introduce path of the images
path='Images/'
images = glob.glob(path+'Run2/*.jpg')

# determine characteristics of the chessboard
size1 = 9
size2 = 6
square_size = 22 #mm

In [ ]:
""""
OFFLINE PHASE: Implementation of the geometric-camera
calibration using the training images from each run.
"""

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, square_size, 0.001)

# create an array of the real world points in mm starting from the top left
objp = np.zeros((size1*size2,3), np.float32)
objp[:,:2] = np.mgrid[0:size1,0:size2].T.reshape(-1,2)
objp[:,:2]=objp[:,:2]*square_size # in mm

# arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space (mm)
imgpoints = [] # 2d points in image plane (pixels).
corner_clicks=[]  # pixel coordinates for manual corners

# iterate through each image
for fname in images:
    img = cv.imread(fname)
    # apply canny filter, edge detection for a sharpened image
    _,img = functions.canny(img)
     # convert image to gray
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (size1, size2), None)
    print(corners)
    count=[]
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        # Detect corners location in subpixels
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (size1,size2), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(30)
    # if not found, do manual selection of corners
    if not ret:
        objpoints.append(objp)
        # Get mouse click coordinates
        cv.setMouseCallback('img', click_corner)
        k=0
        # Close the window when key q is pressed
        while k!=113:
          # Display the image
          cv.imshow("img", img)
          k = cv.waitKey(0)

        # interpolate the corners of the chessboard grid
        corners_manual,corners_print = functions.interpolation(corner_clicks, size1, size2)
        corners_manual=np.array(corners_manual,dtype='float32')

        # empty corner_clicks for next manual image
        corner_clicks=[]

        # Draw the interpolated points on the image
        color = (0, 0, 255) # in BGR format
        radius = 5
        for dot in corners_print:
            cv.circle(img, dot, radius, color, -1)
        # Show the image with the dots
        imgpoints.append(corners_manual)
        cv.imshow('img', img)
        cv.waitKey(1000)

cv.destroyAllWindows()


In [ ]:
# Calibration with all images
print("Total error before the detection and rejection of low quality input images")
_, _, _, _, tot_error = functions.calibration(objpoints, imgpoints, gray)

#Detection of low quality images by detecting outliers in calibration error
outliers, objpoints, imgpoints = functions.reject_outliers(tot_error, objpoints, imgpoints)

# Calibration removing low quality images
print("Total error before the detection and rejection of low quality input images")
mtx, dist, rvecs, tvecs, _ = functions.calibration(objpoints, imgpoints, gray)


# Online phase

## New image calibration

In [ ]:
img = cv.imread(path+'test_def.jpg')
# height and width
h,  w = img.shape[:2]
# obtain the new k matrix of new image based on the estimated instrisic parameters
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# correct test image with estimated parameters from calibration
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite(path+'test_calibrated.jpg', dst)

In [ ]:
img = cv.imread(path+'test_calibrated.jpg')
img = functions.online_phase(img, newcameramtx, dist, size1, size2,square_size)

In [ ]:
# Visualize the final image with the cube and axis
cv.imshow('test',img)
cv.waitKey(8000)
cv.destroyAllWindows()
cv.imwrite(path+'Run2.jpg', img)

### Video Capturing

In [ ]:
# Video capture
cam=cv.VideoCapture(0)
while True:
    hasframe,frame=cam.read()

    # if video is False stop video capturing
    if hasframe==False:
        break
    # for each frame in video obtain cube and axis
    im=functions.online_phase(frame, newcameramtx, dist, size1, size2, square_size)
    cv.imshow('video',im)
    # Close the window when key q is pressed
    if cv.waitKey(1)==113:
        break
cv.destroyAllWindows()
cam.release()
